In [2]:
import pandas as pd
import numpy as np

In [11]:
!pip install tensorflow_text

ERROR: Could not find a version that satisfies the requirement tensorflow_text (from versions: none)
ERROR: No matching distribution found for tensorflow_text


In [3]:
df_m = pd.read_csv('../../gen/output/amazon_search_hedonic_clean_merged.csv', sep = ";")

In [4]:
#select helpful and review
df_m = df_m[['review', 'helpful']]

In [22]:
df_m['helpful'] = (df_m['helpful'] != 0).astype(int)

In [23]:
#save dataset
df_m.to_csv('../../gen/output/review_helpful.csv', sep = ";", index = False)

In [24]:
df_m

,review,helpful
0,This brand DVD player works great. Will be bu...,0
1,This DVD player upscales all our DVD's. Old D...,1
2,5 to 10 second delay in input from remote is a...,0
3,"Like I said a great picture, great value for t...",0
4,This compact DVD player easily fit atop our co...,0
...,...,...
121574,I went through 2 Samsung smart phones in the s...,0
121575,"I've only had it for a couple weeks, but it's ...",0
121576,Don't buy this if you are trying to activate i...,0
121577,"Can't complain about speed, it performs fine.C...",0


In [ ]:
df_m['review'] = df_m['review'].fillna('')

In [ ]:
texts = list(data['review'])
labels = list(data['helpful'])

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_m.drop(columns=['helpful']), df_m['helpful'], test_size=0.6, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [ ]:
#tokenize texts and prepare for bert model
from transformers import BertTokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

def tokenize_sentences(sentences, tokenizer, max_seq_len = 128):
    tokenized_sentences = []

    for sentence in sentences:
        tokenized_sentence = tokenizer.encode(sentence, add_special_tokens = True, max_length = max_seq_len, truncation = True)
        tokenized_sentences.append(tokenized_sentence)

    return tokenized_sentences

def create_attention_masks(tokenized_and_padded_sentences):
    attention_masks = []

    for sentence in tokenized_and_padded_sentences:
        att_mask = [int(token_id > 0) for token_id in sentence]
        attention_masks.append(att_mask)

    return np.array(attention_masks)

def create_model_inputs(df, tokenizer, max_seq_len):
    tokenized_sentences = tokenize_sentences(df['review'], tokenizer, max_seq_len)
    padded_sentences = pad_sequences(tokenized_sentences, maxlen=max_seq_len, dtype="long", value=0, truncating="post", padding="post")
    attention_masks = create_attention_masks(padded_sentences)

    return padded_sentences, attention_masks

max_seq_len = 128

X_train_input = create_model_inputs(X_train, tokenizer, max_seq_len)
X_val_input = create_model_inputs(X_val, tokenizer, max_seq_len)
X_test_input = create_model_inputs(X_test, tokenizer, max_seq_len)

from transformers import TFBertForSequenceClassification

def create_model():
    model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = 2)
    return model

model = create_model()

model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

model.fit(X_train_input, y_train, epochs = 1, batch_size = 32, validation_data = (X_val_input, y_val))

In [ ]:
#install tensorflow_text
import tensorflow as tf


In [ ]:
#make text_dataset from X_train
train_dataset = tf.data.Dataset.from_tensor_slices((X_train['review'].values, y_train.values))


In [12]:
#create  tf.data.Datasets
import tensorflow as tf
#import tensorflow_text as text

X_train = tf.data.Dataset.from_tensor_slices((X_train['review'].values, y_train.values))
X_val = tf.data.Dataset.from_tensor_slices((X_val['review'].values, y_val.values))
X_test = tf.data.Dataset.from_tensor_slices((X_test['review'].values, y_test.values))

In [20]:
#show text and labels
for text, label in X_train.take(3):
    print(text)
    print(label)

tf.Tensor(b'never purchases dvd plyer', shape=(), dtype=string)
tf.Tensor(0.0, shape=(), dtype=float64)
tf.Tensor(b"I'm so glad I invested the inexpensive amount of money for this wonderful, and so easy to install, multi-format player!! It has opened an entire new world of blu-ray and DVDs from other Regions of the world that are actually in English,  copied in other region playing formats. There are some great classics in blu-ray like Long Hot Summer and Love With the Proper Stranger for Paul Newman and Steve McQueen fans, and many more newer films. Some people are purists, like myself, and want Blu-rays. Some are fine with DVDs and that's okay too. For the money and the opportunity to view films I wouldn't be able to otherwise, it's worth every penny!", shape=(), dtype=string)
tf.Tensor(6.0, shape=(), dtype=float64)
tf.Tensor(b"Just got this off the porch where it was delivered. Opened the box, put it together quickly and easily. My mother's downstairs apartment was covered in dog ha